In [15]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

# 이 외에도 어떠한 memory도 사용할 수 있음. 필요에 따라 memory 선택
# ConversationSummaryBuffer: Interaction을 가지고 있는 conversation class
# 설정한 토큰을 넘어서면 이전의 기록은 요약하여 저장함
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    # memory에 있는 내용을 템플릿에 삽입
    # memory.load_memory_variables를 이용하지 않아도 됨
    # ConversationMemory 클래스에게 memory key 속성만 불러주면 됨
    # memory key 이름과 template 내부의 변수명은 동일해야 함
    # 이름은 원하는 무엇이든 상관없음. 두 군데에 같기만 하면 됨.
    memory_key="chat_history",
)

# 템플릿에 메모리에 기억된 내용을 불러와 프롬프트에 전달할 수 있도록 함
template = """
    You are a helpful AI talking to a human

    {chat_history}
    Human: {question}
    You:
"""

# Chain 생성
chain = LLMChain(
    llm=llm,
    memory=memory,
    # 이전 대화 기록이 들어있는 메모리를 가져와 template에 넣어 둠. 그것을 프롬프트에 연결
    prompt=PromptTemplate.from_template(template),
    # verbose는 Chain의 Prompt를 눈으로 확인할 수 있게 해 줌
    verbose=True,
)

chain.predict(question="My name is Vero")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human

    
    Human: My name is Vero
    You:


> Finished chain.


'Hello Vero! How can I assist you today?'

In [16]:
chain.predict(question="I live in Cheonan")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human

    Human: My name is Vero
AI: Hello Vero! How can I assist you today?
    Human: I live in Cheonan
    You:


> Finished chain.


"That's great to know! Is there anything specific you would like help with regarding Cheonan?"

In [17]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human

    Human: My name is Vero
AI: Hello Vero! How can I assist you today?
Human: I live in Cheonan
AI: That's great to know! Is there anything specific you would like help with regarding Cheonan?
    Human: What is my name?
    You:


> Finished chain.


'Your name is Vero.'

In [ ]:
memory.load_memory_variables({})